In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import norm
from scipy.optimize import brentq
from scipy.integrate import quad

# ===== Helper functions given in the assignment =====

def estimateDiscountFactor(row):
    """Estimate discount factor from put-call parity."""
    avgK = row['Strike Price'].mean()
    avgO = (row['Call Premium'] - row['Put Premium']).mean()
    avgKK = (row['Strike Price'] ** 2).mean()
    avgKO = (row['Strike Price'] * (row['Call Premium'] - row['Put Premium'])).mean()
    return (avgKO - avgK * avgO) / (avgK ** 2 - avgKK)

def estimateForwardPrice(row):
    """Estimate forward price from put-call parity."""
    avgK = row['Strike Price'].mean()
    avgO = (row['Call Premium'] - row['Put Premium']).mean()
    avgKK = (row['Strike Price'] ** 2).mean()
    avgKO = (row['Strike Price'] * (row['Call Premium'] - row['Put Premium'])).mean()
    return (avgK * avgKO - avgKK * avgO) / (avgKO - avgK * avgO)

def black_call(F, K, T, sigma, D):
    if sigma <= 0:
        return max(D * (F - K), 0.0)
    st = sigma * np.sqrt(T)
    d1 = np.log(F / K) / st + 0.5 * st
    d2 = d1 - st
    return D * (F * norm.cdf(d1) - K * norm.cdf(d2))

def implied_vol(C_mkt, F, K, T, D):
    def f(sig):
        return black_call(F, K, T, sig, D) - C_mkt
    try:
        return brentq(f, 1e-6, 5)
    except Exception:
        return np.nan

# ===== Data prep for Question 3 =====

# Load midprice data (assumes Midprices.csv in current folder)
df_raw = pd.read_csv("Midprices.csv")

# Parse dates (adjust format if needed)
df_raw['As of Date'] = pd.to_datetime(df_raw['As of Date'])
df_raw['Expiration Date'] = pd.to_datetime(df_raw['Expiration Date'])

# As-of and expiry dates for Question 3
as_of = pd.to_datetime("2024-08-07")
expiry = pd.to_datetime("2024-09-06")

# Filter dataset
df_87_96 = df_raw[
    (df_raw['As of Date'] == as_of) &
    (df_raw['Expiration Date'] == expiry)
].copy()

if df_87_96.empty:
    raise ValueError("No rows found for 8/7/2024 – 9/6/2024 in Midprices.csv")

# Time to expiry (assumed constant within this subset)
T = df_87_96['Time to Expiration'].iloc[0]

# Spot price S0 from underlying
S0 = df_87_96['Underlying Price'].iloc[0]

# Estimate discount factor D(T) and forward F(0,T)
D_T = estimateDiscountFactor(df_87_96)
F_T = estimateForwardPrice(df_87_96)

print("T =", T)
print("S0 =", S0)
print("D(T) =", D_T)
print("F0,T =", F_T)

# Risk-neutral drift r - q implied by F_T and S0
mu_rq = np.log(F_T / S0) / T
print("r - q (mu_rq) =", mu_rq)

# ===== GARCH diffusion parameters (from calibration in Q2) =====

v0   = 0.08364961
v_bar = 0.05127939
lam  = 1.697994
eta  = 8.396695
rho  = -0.6921993  # corr(W, Z)

# ===== Milstein step for v_t =====

def milstein_step_v(v, dt, eps2):
    """
    One Milstein step for v_t under:
        dv_t = lam*(v_bar - v_t) dt + eta * v_t dZ_t
    """
    a = lam * (v_bar - v)
    b = eta * v
    v_new = (
        v
        + a * dt
        + b * np.sqrt(dt) * eps2
        + 0.5 * b * eta * dt * (eps2**2 - 1.0)
    )
    # enforce non-negativity
    return max(v_new, 0.0)

# ===== Joint simulation of (S_t, v_t) with barrier monitoring =====

def simulate_garch_paths_barrier(
    B,              # barrier level
    K,              # strike
    S0,             # spot
    v0, v_bar, lam, eta, rho,
    mu_rq,          # r - q
    T,
    N_paths=5000,
    N_steps=5000,
    D_T=1.0,
    rng=None
):
    """
    Monte Carlo estimate of a down-and-out put in the GARCH model using:
      - Milstein for v_t
      - Euler for log S_t
    Barrier condition: min_t S_t > B
    """
    if rng is None:
        rng = np.random.default_rng(0)

    dt = T / N_steps
    sqrt_dt = np.sqrt(dt)

    payoffs = np.empty(N_paths)

    for i in range(N_paths):
        v = v0
        X = np.log(S0)
        S_min = S0

        for _ in range(N_steps):
            # Correlated standard normals
            z1 = rng.standard_normal()
            z2 = rng.standard_normal()
            eps1 = z1
            eps2 = rho * z1 + np.sqrt(1.0 - rho**2) * z2

            # Milstein update for v
            a = lam * (v_bar - v)
            b = eta * v
            v_new = (
                v
                + a * dt
                + b * sqrt_dt * eps2
                + 0.5 * b * eta * dt * (eps2**2 - 1.0)
            )
            v = max(v_new, 0.0)

            # Euler update for log S
            X += (mu_rq - 0.5 * v) * dt + np.sqrt(max(v, 0.0)) * sqrt_dt * eps1
            S = np.exp(X)

            if S < S_min:
                S_min = S

        # Barrier condition: min_t S_t > B
        if S_min > B:
            payoff = max(K - S, 0.0)
        else:
            payoff = 0.0

        payoffs[i] = payoff

    price_est = D_T * np.mean(payoffs)
    stderr_est = D_T * np.std(payoffs, ddof=1) / np.sqrt(N_paths)
    return price_est, stderr_est

# ===== Run MC for a grid of barrier levels (Q3(a)) =====

K = 5150.0
N_paths = 5000
N_steps = 5000

B_grid = np.linspace(0.0, 5200.0, 41)  # barriers from 0 to 5200
mc_prices = []
mc_stderr = []

for B in B_grid:
    price, err = simulate_garch_paths_barrier(
        B=B,
        K=K,
        S0=S0,
        v0=v0,
        v_bar=v_bar,
        lam=lam,
        eta=eta,
        rho=rho,
        mu_rq=mu_rq,
        T=T,
        N_paths=N_paths,
        N_steps=N_steps,
        D_T=D_T
    )
    mc_prices.append(price)
    mc_stderr.append(err)
    print(f"[GARCH MC] B={B:.1f}, price={price:.6f}, stderr={err:.6f}")

mc_prices = np.array(mc_prices)
mc_stderr = np.array(mc_stderr)

plt.figure()
plt.plot(B_grid, mc_prices, marker='o')
plt.xlabel("Barrier level B")
plt.ylabel("Down-and-out put price (GARCH, Milstein)")
plt.title("Q3(a): Barrier put in GARCH model (MC Milstein)")
plt.grid(True)
plt.show()

# ===== Black–Scholes inputs for Carr–Bowie (Q3(b)) =====

# Choose option with strike closest to K to infer sigma
df_87_96['K_diff'] = (df_87_96['Strike Price'] - K).abs()
row_closest = df_87_96.loc[df_87_96['K_diff'].idxmin()]

K_closest = row_closest['Strike Price']
P_mid_closest = row_closest['Put Premium']
C_mid_closest = row_closest['Call Premium']

print("Closest strike to K:", K_closest)

# Implied vol from call price at K_closest using Black's formula
sigma_bs = implied_vol(
    C_mkt=C_mid_closest,
    F=F_T,
    K=K_closest,
    T=T,
    D=D_T
)

print("Implied volatility sigma_bs =", sigma_bs)

# Recover r and q from D_T and F_T, S0
r = -np.log(D_T) / T
mu_rq_bs = np.log(F_T / S0) / T  # r - q
q = r - mu_rq_bs

print("r =", r, "q =", q)

# ===== Carr–Bowie implementation in Black–Scholes =====

def lognormal_pdf_S(s, S0, r, q, sigma, T):
    """
    Density of S_T under Black–Scholes with continuous r, q.
    """
    if s <= 0:
        return 0.0
    mu = np.log(S0) + (r - q - 0.5 * sigma**2) * T
    var = sigma**2 * T
    return (1.0 / (s * np.sqrt(2.0 * np.pi * var))) * np.exp(
        -(np.log(s) - mu)**2 / (2.0 * var)
    )

def H_put(s, K):
    """Vanilla put payoff (K - s)+."""
    return max(K - s, 0.0)

def H_tilde_carr_bowie(s, K, B, r, sigma):
    """
    Image payoff \tilde H(s) as in Lecture 5 (upper barrier case):
        Htilde(x) = (x/B)^{1 - 2r / sigma^2} * H(B^2 / x)
    """
    if s <= 0:
        return 0.0
    alpha = 1.0 - 2.0 * r / (sigma**2)
    return (s / B)**alpha * H_put(B**2 / s, K)

def carr_bowie_barrier_put_bs(K, B, S0, r, q, sigma, T, n_std=8):
    """
    Carr–Bowie barrier put price in Black–Scholes via numerical integration:
        Price = e^{-rT} * (E[H(S_T)] - E[Htilde(S_T)]).
    """
    D_T = np.exp(-r * T)

    mu = np.log(S0) + (r - q - 0.5 * sigma**2) * T
    std = sigma * np.sqrt(T)

    y_min = mu - n_std * std
    y_max = mu + n_std * std

    # Change variable y = log s
    def integrand_H(y):
        s = np.exp(y)
        return H_put(s, K) * lognormal_pdf_S(s, S0, r, q, sigma, T) * np.exp(y)

    def integrand_Htilde(y):
        s = np.exp(y)
        return H_tilde_carr_bowie(s, K, B, r, sigma) * lognormal_pdf_S(s, S0, r, q, sigma, T) * np.exp(y)

    E_H, _ = quad(integrand_H, y_min, y_max, epsabs=1e-7, epsrel=1e-7)
    E_Htilde, _ = quad(integrand_Htilde, y_min, y_max, epsabs=1e-7, epsrel=1e-7)

    price = D_T * (E_H - E_Htilde)
    return price

# ===== Sweep barrier levels in BS/Carr–Bowie and compare with GARCH MC =====

bs_prices = []

for B in B_grid:
    price_bs = carr_bowie_barrier_put_bs(
        K=K,
        B=B,
        S0=S0,
        r=r,
        q=q,
        sigma=sigma_bs,
        T=T
    )
    bs_prices.append(price_bs)
    print(f"[BS Carr-Bowie] B={B:.1f}, price={price_bs:.6f}")

bs_prices = np.array(bs_prices)

plt.figure()
plt.plot(B_grid, mc_prices, marker='o', label="GARCH (Milstein MC)")
plt.plot(B_grid, bs_prices, marker='x', linestyle='--', label="BS (Carr–Bowie)")
plt.xlabel("Barrier level B")
plt.ylabel("Barrier put price")
plt.title("Q3(a,b): Down-and-out put prices vs barrier level")
plt.legend()
plt.grid(True)
plt.show()


T = 0.0833333333333333
S0 = 5202.9406827025
D(T) = 0.9951226352831437
F0,T = 5219.875354207246
r - q (mu_rq) = 0.03899449748691378
[GARCH MC] B=0.0, price=237.550890, stderr=5.650049
[GARCH MC] B=130.0, price=237.550890, stderr=5.650049
[GARCH MC] B=260.0, price=236.580599, stderr=5.574355
[GARCH MC] B=390.0, price=236.580599, stderr=5.574355
[GARCH MC] B=520.0, price=236.580599, stderr=5.574355
[GARCH MC] B=650.0, price=236.580599, stderr=5.574355
[GARCH MC] B=780.0, price=235.803465, stderr=5.526555
[GARCH MC] B=910.0, price=235.803465, stderr=5.526555
[GARCH MC] B=1040.0, price=235.803465, stderr=5.526555
[GARCH MC] B=1170.0, price=235.138339, stderr=5.492084
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Ruotong Guan\anaconda3\envs\adm\Lib\site-packages\IPython\core\interactiveshell.py", line 3699, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Ruotong Guan\AppData\Local\Temp\ipykernel_46032\88531181.py", line 191, in <module>
    price, err = simulate_garch_paths_barrier(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Ruotong Guan\AppData\Local\Temp\ipykernel_46032\88531181.py", line None, in simulate_garch_paths_barrier
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Ruotong Guan\anaconda3\envs\adm\Lib\site-packages\IPython\core\interactiveshell.py", line 2194, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Ruotong Guan\anaconda3\envs\adm\Lib\site-packages\IPython\core\ultratb.py", line 1188, in structu